# Cost analysis on trips to US and Europe

In [1]:
import requests
import json
import os
import glob
from os.path import basename
from os.path import splitext
import datetime
import calendar
import pandas as pd
import plotly.plotly as py
import plotly.graph_objs as go

#api_key = "api-key=" + os.getenv("nyt_archive_key")

In [2]:
flights = pd.read_csv('../Output/BostonFlightsData.csv', low_memory=False)

In [3]:
sky_domain = 'http://partners.api.skyscanner.net/apiservices/browsequotes/v1.0/US/USD/en-US/'
sky_api_key = 'apiKey=' +'su273465397115788107773117889676'
def geturl(source, dest, outb_date="__", inb_date="__"):    
    return sky_domain + source + "/" + dest + "/" + outb_date + "/" + inb_date + "?" + sky_api_key 

In [ ]:
flights['URL'] = flights.apply(lambda x: geturl(x.Source_Airport, x.Destination_Airport) , axis=1)

In [14]:
def add_months(sourcedate,months):
    month = sourcedate.month - 1 + months
    year = int(sourcedate.year + month / 12 )
    month = month % 12 + 1
    day = min(sourcedate.day, calendar.monthrange(year,month)[1])
    return datetime.date(year,month,day)


d  = datetime.date.today()
while d.weekday() != 4:
    d += datetime.timedelta(1)

start_date = d
end_date = add_months(start, 5)

datetime.date(2017, 9, 21)

In [18]:
depart_dates = []
return_dates = []
delta = datetime.timedelta(days=1)
d = start_date
weekend = set([4, 6])
while d <= end_date:
    if d.weekday() == 4:
        depart_dates.append(d)
    elif d.weekday() == 6:
        return_dates.append(d)
    d += delta

In [24]:
def createdir(path): # Function to create directory if it does not exist already
    try:
        if not os.path.exists(path):
            os.makedirs(path)
        return True
    except OSError as exception:
        return False

download_dir = "../Output/QuotesData/"
val = createdir(download_dir)

def checkofflinedata():
    if os.path.exists(download_dir):
        return True
    else:
        return False

In [25]:
def DownloadData():
    if (checkofflinedata == False):
        for _, rows in flights.iterrows():    
            outdir = download_dir + rows.Source_Airport + "_" + rows.Destination_Airport
            createdir(outdir)
            for dep, arr in zip(depart_dates, return_dates):
                url = geturl(rows.Source_Airport, rows.Destination_Airport, dep.strftime("%Y-%m-%d"), arr.strftime("%Y-%m-%d"))
                response = requests.get(url)
                if response.status_code is 200:
                    outputdir = outdir + "/" + str(dep.year) + "/" + str(dep.month)
                    createdir(outputdir)
                    try:
                        with open(outputdir + "/" + str(dep.isocalendar()[1]) + ".json", "w") as jsonfile:
                            json.dump(response.json(), jsonfile, indent=4, sort_keys=True)
                    except:
                        print("Failed in dumping {0}".format(outputdir + "/" + str(dep.isocalendar()[1]) + ".json"))
                else:
                    print("Are you online? If yes, then your url seems to be fuzzy or skyscanner went down under")

DownloadData()

In [26]:
quotes_df = pd.DataFrame()
for _, rows in flights.iterrows():
    outdir = download_dir + rows.Source_Airport + "_" + rows.Destination_Airport
    weeks = []
    avg_price = []
    d = start_date
    while d <= end_date:
        outputdir = outdir + "/" + str(d.year) + "/" + str(d.month)
        if os.path.exists(outputdir):
            json_files = glob.glob(outputdir + '/*.json')
            if json_files is not None:
                for i in range(len(json_files)):
                    min_price = 0
                    cnt = cnt_d = cnt_r = 0
                    dep_price = ret_price = 0
                    with open(json_files[i]) as datafile:
                        quote_data = json.load(datafile)
                        for x in range(0, len(quote_data["Quotes"])):
                            json_data = quote_data["Quotes"][x]
                            try :
                                if "OutboundLeg" in json_data and "InboundLeg" in json_data: 
                                    if json_data["OutboundLeg"] and json_data["InboundLeg"] is not None:                                
                                        min_price += json_data["MinPrice"]
                                        cnt += 1
                                        continue;
                                if "OutboundLeg" in json_data:
                                    if json_data["OutboundLeg"] is not None:
                                        dep_price += json_data["MinPrice"]
                                        cnt_d += 1
                                if "InboundLeg" in json_data:
                                    if json_data["InboundLeg"] is not None:
                                        ret_price += json_data["MinPrice"]
                                        cnt_r += 1
                                    
                            except:
                                print(json_files[i])
                                
                        base = basename(json_files[i])
                        weeks.append(str(d.year)+ '-' +splitext(base)[0])
                        if cnt_d > 0 and cnt_r > 0 and cnt_d == cnt_r:
                            min_price += (dep_price/cnt_d) + (ret_price/cnt_r)
                            cnt += ((cnt_d + cnt_r)/ 2)
                        if cnt != 0:
                            avg_price.append((min_price/cnt))
                        else:
                            avg_price.append(0)
                            
        d += datetime.timedelta(days=31)
    
    df = pd.DataFrame({"Weekend_Date" : weeks, "Average_Price" : avg_price, "Source" : rows.Source_Airport,
                       "Destination" : rows.Destination_Airport})
    quotes_df = quotes_df.append(df)

In [28]:
quotes_df.to_csv("../Output/QuotesOnUSDestinations.csv", index=False)
quotes_df.head()

,Average_Price,Destination,Source,Weekend_Date
0,553.0,MSP,BOS,2017-16
1,246.0,MSP,BOS,2017-17
2,246.0,MSP,BOS,2017-18
3,266.0,MSP,BOS,2017-19
4,265.0,MSP,BOS,2017-20


In [30]:
bos_den = quotes_df[quotes_df['Destination'] == 'DEN']
bos_den['Weekend_Date'] = bos_den.apply(lambda x: datetime.datetime.strptime(x.Weekend_Date + '-0', "%Y-%W-%w"), axis=1)
bos_den['Average_Price'] = bos_den.apply(lambda x: bos_atl['Average_Price'].mean() 
                                         if x.Average_Price == 0 else x.Average_Price, axis=1)

/Users/sumitdeshmukh/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/sumitdeshmukh/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [31]:
bos_lax = quotes_df[quotes_df['Destination'] == 'LAX']
bos_lax['Weekend_Date'] = bos_lax.apply(lambda x: datetime.datetime.strptime(x.Weekend_Date + '-0', "%Y-%W-%w"), axis=1)
bos_lax['Average_Price'] = bos_lax.apply(lambda x: bos_lax['Average_Price'].mean() 
                                         if x.Average_Price == 0 else x.Average_Price, axis=1)

/Users/sumitdeshmukh/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/sumitdeshmukh/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [32]:
bos_sfo = quotes_df[quotes_df['Destination'] == 'SFO']
bos_sfo['Weekend_Date'] = bos_sfo.apply(lambda x: datetime.datetime.strptime(x.Weekend_Date + '-0', "%Y-%W-%w"), axis=1)
bos_sfo['Average_Price'] = bos_sfo.apply(lambda x: bos_sfo['Average_Price'].mean() 
                                         if x.Average_Price == 0 else x.Average_Price, axis=1)

/Users/sumitdeshmukh/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/sumitdeshmukh/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [33]:
bos_iah = quotes_df[quotes_df['Destination'] == 'IAH']
bos_iah['Weekend_Date'] = bos_iah.apply(lambda x: datetime.datetime.strptime(x.Weekend_Date + '-0', "%Y-%W-%w"), axis=1)
bos_iah['Average_Price'] = bos_iah.apply(lambda x: bos_iah['Average_Price'].mean() 
                                         if x.Average_Price == 0 else x.Average_Price, axis=1)

/Users/sumitdeshmukh/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/sumitdeshmukh/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [34]:
bos_phx = quotes_df[quotes_df['Destination'] == 'PHX']
bos_phx['Weekend_Date'] = bos_phx.apply(lambda x: datetime.datetime.strptime(x.Weekend_Date + '-0', "%Y-%W-%w"), axis=1)
bos_phx['Average_Price'] = bos_phx.apply(lambda x: bos_phx['Average_Price'].mean() 
                                         if x.Average_Price == 0 else x.Average_Price, axis=1)

/Users/sumitdeshmukh/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/sumitdeshmukh/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [38]:
Data = [
        go.Scatter(
            x = bos_den.Weekend_Date,
            y = bos_den.Average_Price,
            name = 'Denver'
        ),
        go.Scatter(
            x = bos_lax.Weekend_Date,
            y = bos_lax.Average_Price,
            name = 'Los Angeles'
        ),
        go.Scatter(
            x = bos_sfo.Weekend_Date,
            y = bos_sfo.Average_Price,
            name = 'San Francisco'
        ),
        go.Scatter(
            x = bos_iah.Weekend_Date,
            y = bos_iah.Average_Price,
            name = 'Houston'
        ),
        go.Scatter(
            x = bos_phx.Weekend_Date,
            y = bos_phx.Average_Price,
            name = 'Phoenix'
        )]

layout = go.Layout(             
    title="Weekend Return Trip Prices for major US cities from Boston", 
    xaxis=dict(                 
        title="Weekend dates"            
    ),
    yaxis= dict(
        title="Return Ticket price in USD"
    ),
)

figure = go.Figure(data=data, layout=layout)

py.iplot(figure, filename='Weekend Return Trip Prices for major US cities from Boston')